In [2]:
# Step 1: Generate conventional tetragonal LLZO unit cell
from pymatgen.core import Lattice, Structure

# Define lattice parameters (Å)
a = b = 13.236
c = 12.702
lattice = Lattice.from_parameters(a, b, c, 90, 90, 90)

# Define atomic species and fractional coordinates (from Wyckoff positions)
species = [
    "La",  # 8b
    "La",  # 16e
    "Zr",  # 16c
    "Li",  # 8a
    "Li",  # 16f
    "Li",  # 32g
    "O",   # 32g
    "O",   # 32g
    "O"    # 32g
]
coords = [
    [0.0000, 0.2500, 0.1250],   # La (8b)
    [0.12716, 0.0000, 0.2500],  # La (16e)
    [0.0000, 0.0000, 0.0000],   # Zr  (16c)
    [0.0000, 0.2500, 0.3750],   # Li  (8a)
    [0.1813,  0.4313, 0.1250],   # Li  (16f)
    [0.0796,  0.0863, 0.8099],   # Li  (32g)
    [-0.0335, 0.0546, 0.1528],   # O   (32g)
    [0.0534,  0.8525, 0.5366],   # O   (32g)
    [0.1499,  0.0273, 0.4454]    # O   (32g)
]

# Build the structure
unit_cell = Structure(lattice, species, coords, coords_are_cartesian=False)

# Write to CIF for verification
unit_cell.to("LLZO_tetragonal_unit_cell.cif", "cif")
print("Step 1 complete: LLZO unit cell saved as 'LLZO_tetragonal_unit_cell.cif'")


Step 1 complete: LLZO unit cell saved as 'LLZO_tetragonal_unit_cell.cif'


In [3]:
# Step 2: Replicate the LLZO unit cell in the z-direction using ASE

from ase.io import read, write
from ase.build import make_supercell
import numpy as np

# Load the unit cell CIF
unit = read("LLZO_tetragonal_unit_cell.cif")

# Determine replication factor: ceil(15 Å / 12.702 Å) = 2
rep_z = 2

# Build supercell matrix: 1×1×2
supercell_mat = np.diag([1, 1, rep_z])

# Create supercell
slab_supercell = make_supercell(unit, supercell_mat)

# Write out the new CIF
write("LLZO_slab_supercell_1x1x2.cif", slab_supercell)
print("Step 2 complete: Saved 1×1×2 supercell as 'LLZO_slab_supercell_1x1x2.cif'")


Step 2 complete: Saved 1×1×2 supercell as 'LLZO_slab_supercell_1x1x2.cif'


In [4]:
# Step 3: Center the slab within the cell along the z-axis for symmetric surfaces

from ase.io import read, write

# Load the replicated slab
slab = read("LLZO_slab_supercell_1x1x2.cif")

# Center atoms along z without adding vacuum (align slab mid-point at half cell height)
slab.center(vacuum=0, axis=2)

# Wrap any atoms outside [0,1] fractional coordinates back into the cell
slab.wrap()

# Save the centered slab
write("LLZO_slab_centered.cif", slab)
print("Step 3 complete: Saved centered slab as 'LLZO_slab_centered.cif'")


Step 3 complete: Saved centered slab as 'LLZO_slab_centered.cif'


In [9]:
# Step 4 (Pymatgen version): Trim slab to retain Li₀.₅La₀.₅ZrO₂-terminated surfaces

from pymatgen.core import Structure
import numpy as np

# Load centered supercell from Step 3
slab = Structure.from_file("LLZO_slab_centered.cif")

# Get all Zr atom z-coordinates (fractional)
zr_z = sorted([site.frac_coords[2] for site in slab if site.specie.symbol == "Zr"])

# Compute trim bounds: halfway between 1st–2nd and (n−2)–(n−1) Zr layers
low_cut = (zr_z[0] + zr_z[1]) / 2
high_cut = (zr_z[-2] + zr_z[-1]) / 2

# Filter sites between those bounds
trimmed_sites = [site for site in slab if low_cut <= site.frac_coords[2] <= high_cut]

# Build new Structure
trimmed_slab = Structure(
    lattice=slab.lattice,
    species=[site.specie for site in trimmed_sites],
    coords=[site.frac_coords for site in trimmed_sites],
    coords_are_cartesian=False
)

# Write to CIF
trimmed_slab.to("LLZO_slab_trimmed.cif", "cif")
print("Step 4 complete: Trimmed symmetric slab saved as 'LLZO_slab_trimmed.cif'")


IndexError: tuple index out of range